In [1]:
import os
import numpy as np
import pandas as pd

from exp_spec_info import *
from plot_info import *
from select_data import *

In [2]:
# Data pickle path
extracted_data_path = "C:\\Users\\dosre\\dev\\thesis-data\\extracted_data.pkl"

# Processed data paths
extended_data_path = "C:\\Users\\dosre\\dev\\thesis-data\\extended_data.pkl"
median_data_path = "C:\\Users\\dosre\\dev\\thesis-data\\median_data.pkl"

##### Extend Data with Processed Elements

In [3]:
data = pd.read_pickle(extracted_data_path)

# Extend with final relative residual
rel_res_ext = []
for setup in SETUPS:
    for matrix in SETUP_MATRIX_MAPPING[setup]:
        for restart_param in RESTART_PARAMS:
            for exp_iter in range(N_EXPERIMENT_ITERATIONS):

                sub_data = df_sel_setup_matrix_restart(
                    data, setup, matrix, restart_param
                )
                sub_data = sub_data[sub_data["experiment_iter"] == exp_iter]

                rel_res = sub_data["inner_relres"].apply(
                    lambda relres_arr: relres_arr[-1]
                )
                if len(rel_res) > 0:
                    rel_res_ext.append(rel_res)

rel_res_ext = pd.concat(rel_res_ext)
rel_res_ext.name = "final_relres"

extended_data = data.join((rel_res_ext))

def integrate_relres_diff_from_baseline(
    target_relres: np.array, baseline_relres: np.array
) -> float:
    target_n = len(target_relres)
    baseline_n = len(baseline_relres)
    if target_n > baseline_n:
        expanded_baseline = np.hstack(
            [baseline_relres, np.full(target_n-baseline_n, baseline_relres[-1])]
        )
        return np.sum(target_relres - expanded_baseline)
    else:
        return np.sum(target_relres - baseline_relres[:target_n])

# Extend with values based on FP64
rel_time_ext = []
rel_res_frac_err_ext = []
outer_integr_fp64_traj_diff_per_iter_ext = []
inner_integr_fp64_traj_diff_per_iter_ext = []
for setup in SETUPS:
    for matrix in SETUP_MATRIX_MAPPING[setup]:
        for restart_param in RESTART_PARAMS:
            for exp_iter in range(N_EXPERIMENT_ITERATIONS):

                sub_data = df_sel_setup_matrix_restart(
                    extended_data, setup, matrix, restart_param
                )
                sub_data = sub_data[sub_data["experiment_iter"] == exp_iter]

                if sub_data[sub_data["solver"] == "FP FP64"].size != 0:

                    fp64_row = sub_data[sub_data["solver"] == "FP FP64"].iloc[0]

                    fp64_time = fp64_row["elapsed_time_ms"]
                    rel_time = sub_data["elapsed_time_ms"]/fp64_time
                    rel_time_ext.append(rel_time)

                    fp64_relres = fp64_row["final_relres"]
                    rel_res_frac_err = (
                        (sub_data["final_relres"]-fp64_relres)/fp64_relres
                    )
                    rel_res_frac_err_ext.append(rel_res_frac_err)

                    fp64_outer_relres = fp64_row["outer_relres"]
                    outer_integr_fp64_traj_diff_per_iter = (
                        sub_data["outer_relres"].transform(
                            integrate_relres_diff_from_baseline,
                            0,
                            fp64_outer_relres
                        )
                    )
                    outer_integr_fp64_traj_diff_per_iter /= sub_data["outer_iters"]
                    assert outer_integr_fp64_traj_diff_per_iter[
                        sub_data["solver"] == "FP FP64"
                    ].values[0] == 0.
                    outer_integr_fp64_traj_diff_per_iter_ext.append(
                        outer_integr_fp64_traj_diff_per_iter
                    )

                    fp64_inner_relres = fp64_row["inner_relres"]
                    inner_integr_fp64_traj_diff_per_iter = (
                        sub_data["inner_relres"].transform(
                            integrate_relres_diff_from_baseline,
                            0,
                            fp64_inner_relres
                        )
                    )
                    inner_integr_fp64_traj_diff_per_iter /= sub_data["inner_iters"]
                    assert inner_integr_fp64_traj_diff_per_iter[
                        sub_data["solver"] == "FP FP64"
                    ].values[0] == 0.
                    inner_integr_fp64_traj_diff_per_iter_ext.append(
                        inner_integr_fp64_traj_diff_per_iter
                    )

rel_time_ext = pd.concat(rel_time_ext)
rel_time_ext.name = "rel_time"

rel_res_frac_err_ext = pd.concat(rel_res_frac_err_ext)
rel_res_frac_err_ext.name = "rel_res_frac_err"

outer_integr_fp64_traj_diff_per_iter_ext = pd.concat(outer_integr_fp64_traj_diff_per_iter_ext)
outer_integr_fp64_traj_diff_per_iter_ext.name = "outer_integr_fp64_traj_diff_per_iter"

inner_integr_fp64_traj_diff_per_iter_ext = pd.concat(inner_integr_fp64_traj_diff_per_iter_ext)
inner_integr_fp64_traj_diff_per_iter_ext.name = "inner_integr_fp64_traj_diff_per_iter"

extended_data = extended_data.join(
    (
        rel_time_ext,
        rel_res_frac_err_ext,
        outer_integr_fp64_traj_diff_per_iter_ext,
        inner_integr_fp64_traj_diff_per_iter_ext
    )
)
display(extended_data)

extended_data.to_pickle(extended_data_path)

,setup,matrix,solver,restart_param,experiment_iter,populated,initiated,converged,terminated,outer_iters,...,HS_trans_inner_iter,SD_trans_outer_iter,SD_trans_inner_iter,outer_relres,inner_relres,final_relres,rel_time,rel_res_frac_err,outer_integr_fp64_traj_diff_per_iter,inner_integr_fp64_traj_diff_per_iter
4080,ilutp1em2,af23560,FP FP16,10,0,True,True,False,True,1500,...,-1,-1,-1,"[1.0, 0.12240884161261857, 0.03422273561924297...","[1.0, 1.157900925472873, 0.7205927713142924, 0...",5.247287e-07,103.182815,524727.730116,6.099064e-06,4.356123e-06
4081,ilutp1em2,af23560,FP FP16,10,1,True,True,False,True,1500,...,-1,-1,-1,"[1.0, 0.12200900834319824, 0.03265599746967163...","[1.0, 1.1281593633462907, 0.6993294403121761, ...",5.288204e-07,103.197688,528819.361770,4.066356e-06,2.760776e-06
4082,ilutp1em2,af23560,FP FP16,10,2,True,True,False,True,1500,...,-1,-1,-1,"[1.0, 0.1211365949289795, 0.031721622859388454...","[1.0, 1.0871766587196103, 0.7007430796515004, ...",5.181185e-07,103.251186,518117.530842,4.657648e-06,2.970493e-06
4104,ilutp1em2,af23560,FP FP32,10,0,True,True,True,True,14,...,-1,-1,-1,"[1.0, 0.11734933791282731, 0.03215894871299636...","[1.0, 1.1575543836183488, 0.7210134584324849, ...",1.000000e-12,0.958222,0.000000,-1.006046e-08,-8.923861e-09
4105,ilutp1em2,af23560,FP FP32,10,1,True,True,True,True,14,...,-1,-1,-1,"[1.0, 0.11869397039884688, 0.03146533932009303...","[1.0, 1.1296459986651255, 0.6990106966195974, ...",1.000000e-12,0.962063,0.000000,-1.984611e-09,-5.049047e-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36475,unprecond,Zhao2,PC HSD S2T,150,1,True,True,True,True,27,...,2102,17,2252,"[1.0, 0.0035827563693714664, 0.001680961968812...","[1.0, 0.9798814359645774, 0.4454125543127461, ...",1.000000e-12,1.031818,0.000000,8.466728e-05,3.843343e-05
36476,unprecond,Zhao2,PC HSD S2T,150,2,True,True,True,True,27,...,2102,17,2252,"[1.0, 0.0035542551630105045, 0.001682663103450...","[1.0, 0.9808629620107184, 0.4513029714477763, ...",1.000000e-12,1.033342,0.000000,7.829484e-05,3.796950e-05
36477,unprecond,Zhao2,PC HSD S2T,200,0,True,True,True,True,20,...,2401,14,2601,"[1.0, 0.0033437792072247615, 0.000694396526344...","[1.0, 0.9766142994493332, 0.44873103183878926,...",1.000000e-12,1.191292,0.000000,1.053429e-04,5.934494e-05
36478,unprecond,Zhao2,PC HSD S2T,200,1,True,True,True,True,19,...,2201,13,2401,"[1.0, 0.005431643250731864, 0.0008261741026476...","[1.0, 0.9779367541164282, 0.4471612329439548, ...",1.000000e-12,1.137896,0.000000,2.292243e-04,6.738861e-05


##### Compress into Experiment Iteration Median Values

In [4]:
median_data = []

for setup in SETUPS:
    for matrix in SETUP_MATRIX_MAPPING[setup]:
        for restart_param in RESTART_PARAMS:

            sub_data = df_sel_setup_matrix_restart(
                extended_data, setup, matrix, restart_param
            )

            for solver in SOLVERS:

                solve_experiments = sub_data[sub_data["solver"] == solver]

                if len(solve_experiments) > 0:

                    median_experiment = {
                        "setup": setup,
                        "matrix": matrix,
                        "solver": solver,
                        "restart_param": restart_param,
                        "all_populated": bool(solve_experiments["populated"].all()),
                        "all_initiated": bool(solve_experiments["initiated"].all()),
                        "all_converged": bool(solve_experiments["converged"].all()),
                        "all_terminated": bool(solve_experiments["terminated"].all()),
                        "med_outer_iter": int(solve_experiments["outer_iters"].median()),
                        "med_inner_iter": int(solve_experiments["inner_iters"].median()),
                        "med_time_ms": int(solve_experiments["elapsed_time_ms"].median()),
                        "med_rel_time": solve_experiments["rel_time"].median(),
                        "med_rel_res": solve_experiments["final_relres"].median(),
                        "med_rel_res_frac_err": solve_experiments["rel_res_frac_err"].median(),
                        "med_outer_integr_fp64_traj_diff_per_iter": solve_experiments["outer_integr_fp64_traj_diff_per_iter"].median(),
                        "med_inner_integr_fp64_traj_diff_per_iter": solve_experiments["inner_integr_fp64_traj_diff_per_iter"].median()
                    }

                    median_data.append(median_experiment)

median_data = pd.DataFrame(median_data)
display(median_data)

median_data.to_pickle(median_data_path)

,setup,matrix,solver,restart_param,all_populated,all_initiated,all_converged,all_terminated,med_outer_iter,med_inner_iter,med_time_ms,med_rel_time,med_rel_res,med_rel_res_frac_err,med_outer_integr_fp64_traj_diff_per_iter,med_inner_integr_fp64_traj_diff_per_iter
0,unprecond,af_0_k101,FP FP16,10,True,True,False,True,1500,15000,159474,0.993768,0.000013,1.344918e+00,4.371271e-06,4.315035e-06
1,unprecond,af_0_k101,FP FP32,10,True,True,False,True,1500,15000,122252,0.761818,0.000005,-2.537117e-02,-2.530822e-08,-2.526722e-08
2,unprecond,af_0_k101,FP FP64,10,True,True,False,True,1500,15000,160457,1.000000,0.000005,0.000000e+00,0.000000e+00,0.000000e+00
3,unprecond,af_0_k101,PC SD ORC,10,True,True,False,True,1500,15000,137222,0.855195,0.000005,-9.946886e-03,-1.390409e-08,-1.388805e-08
4,unprecond,af_0_k101,PC SD CS,10,True,True,False,True,1500,15000,121514,0.760301,0.000005,-2.537117e-02,-2.530822e-08,-2.526722e-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7315,ilutp1em4,Zhao2,PC SD RRT,10,True,True,False,True,1500,1500,90768,0.242435,1.000000,-3.138725e-01,-4.574551e-01,-4.596365e-01
7316,ilutp1em4,Zhao2,PC HSD ORC,10,True,True,False,True,1500,6900,205132,0.547614,1.457455,1.671064e-12,-2.744730e-01,-5.966805e-02
7317,ilutp1em4,Zhao2,PC HSD CS,10,True,True,False,True,1500,14982,374682,1.000243,1.457455,-3.613029e-14,-6.099401e-04,-6.106729e-05
7318,ilutp1em4,Zhao2,PC HSD RRT,10,True,True,False,True,1500,1500,91475,0.244222,1.000000,-3.138725e-01,-4.574551e-01,-4.596365e-01
